In [1]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
from PIL import Image, ImageOps
import torchvision.transforms as transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# DataSets

In [3]:
benchmarkSet = !ls superResolutionSamples/benchmark/*/HR/*

# RefVAE

In [4]:
# RefVAE
from RefVAE.network import encoder4 as refVaeEnc4
from RefVAE.network import decoder4 as refVaeDec4
from RefVAE.model import VAE_v3_4x, F

In [5]:
class RefVae:
    def __init__(self, up_factor=4, 
                 encoder_dir = './RefVAE/models/vgg_r41.pth', 
                 decoder_dir = './RefVAE/models/dec_r41.pth', 
                 model_dir = './RefVAE/models/ConVAE_8x.pth'):
        model = VAE_v3_4x(up_factor=up_factor)
        enc = refVaeEnc4()
        dec = refVaeDec4()
        self.scale_factor = up_factor

        if os.path.exists(encoder_dir):
            enc.load_state_dict(torch.load(encoder_dir))
            print('encoder model is loaded!')

        if os.path.exists(decoder_dir):
            dec.load_state_dict(torch.load(decoder_dir))
            print('decoder model is loaded!')

        for param in enc.parameters():
            param.requires_grad = False

        for param in dec.parameters():
            param.requires_grad = False
            
        model = model.to(device)
        enc = enc.to(device)
        dec = dec.to(device)
        
        if os.path.exists(model_dir):
            print('Loading model')
            model.load_state_dict(torch.load(model_dir, map_location=lambda storage, loc: storage))
            print(model_dir)
        self.enc = enc
        self.dec = dec
        self.model = model
        self.transform = transforms.Compose([
            transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
        ])
        self.style_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
        ])
        
    def predict(self, img):
        with torch.no_grad():
            if type(img) == str:
                img = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
            img = self.transform(img).unsqueeze(0).to(device)
            lr_feat = self.enc(F.interpolate(img, scale_factor=self.scale_factor,
                                            mode='bicubic'))
            SR, _ = self.model(img, lr_feat['r41'])
            del img
            torch.cuda.empty_cache()
            return SR

In [6]:
abc = RefVae()

encoder model is loaded!
Loading model
./RefVAE/models/ConVAE_8x.pth


In [9]:
img = cv2.cvtColor(cv2.imread(benchmarkSet[0]), cv2.COLOR_BGR2RGB)
# img = abc.transform(img).unsqueeze(0)
img.shape

(481, 321, 3)

In [10]:
abc.predict(img)

RuntimeError: CUDA out of memory. Tried to allocate 604.00 MiB (GPU 0; 5.81 GiB total capacity; 4.06 GiB already allocated; 398.88 MiB free; 4.19 GiB reserved in total by PyTorch)

In [47]:
j = abc.enc(F.interpolate(img, scale_factor=4,mode='bicubic'))

RuntimeError: CUDA out of memory. Tried to allocate 604.00 MiB (GPU 0; 5.81 GiB total capacity; 4.27 GiB already allocated; 246.88 MiB free; 4.34 GiB reserved in total by PyTorch)

In [48]:
SR, _ = abc.model(img, j['r41'])

TypeError: conv2d() received an invalid combination of arguments - got (NoneType, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!NoneType!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!NoneType!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)


In [30]:
type('asda') == str

True